In [6]:
import tweepy
import json
import numpy as np
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
consumer_key = '30Z8Dwqy7gkJ1mxOAy1Dqrnmv'
consumer_secret = 'WkI2fvAjFTmFq4LH7aXKHPiB5sAAo2lQDll30gCA3nFscY2WWm'
access_token = '967430440335413249-C7zFPWxyXopWjtUBmD4mVpp14JE9yjX'
access_token_secret = 'iddKLYODgDXvqNxiu1lrxGp5ugfWvQBIw9P1jqr14xLFo'

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [11]:
users=('@BBC', '@CBS', '@CNN', '@foxnews', '@nytimes')
df=({'@BBC':[], '@CBS':[], '@CNN':[], '@foxnews':[], '@nytimes':[]})

In [33]:
oldest_tweet = ""
for user in users
    for x in range(5):
        

{'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Mar 05 18:38:04 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/970730178057039877',
    'indices': [117, 140],
    'url': 'https://t.co/1UBRGqIOp6'}],
  'user_mentions': []},
 'favorite_count': 15,
 'favorited': False,
 'geo': None,
 'id': 970730178057039877,
 'id_str': '970730178057039877',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 8,
 'retweeted': False,
 'source': '<a href="http://www.radian6.com" rel="nofollow">Radian6 -Social Media Management</a>',
 'text': 'No meat. No fish. No eggs. No milk. No cheese. How hard is it to go vegan for a month? \n\n\U0001f966🥑🍏 Nesta Mc